In [1]:
import pandas as pd
import teradataml as tdml

# check if data already exists

In [2]:
complaints_uploaded = False

In [ ]:
try:
    DF_complaints = tdml.DataFrame("telco_consumer_complaints_t")
    print("telco_consumer_complaints_t already exists")
    if DF_complaints.shape == (7023, 771):
        complaints_uploaded = True
        print("telco_consumer_complaints_t has got the right shape")
        
except:
    pass
    

# read and upload complaints

In [ ]:
if not complaints_uploaded:
    df_complaints = pd.read_parquet("data/embeddings.parquet")

    print("start upload: telco_consumer_complaints_t")
    tdml.copy_to_sql(df_complaints, "telco_consumer_complaints_t", 
                     types = {"txt":tdml.VARCHAR(10000),
                             "complaint_date":tdml.DATE(),
                             },
                     primary_index="row_id",
                     if_exists="fail"
                    )
    print("finished upload: telco_consumer_complaints_t")

In [ ]:
tdml.execute_sql("""REPLACE VIEW telco_consumer_complaints AS 
SELECT 
    row_id, 
    complaint_date AS date_received, 
    txt AS consumer_complaint_narrative
FROM telco_consumer_complaints_t
""")

In [ ]:
tdml.execute_sql("""REPLACE VIEW telco_consumer_embeddings AS 
SELECT * FROM Antiselect(
    ON telco_consumer_complaints_t
    USING
    Exclude ('txt', 'complaint_date')
    ) dt
""")